In [1]:
from textblob import TextBlob
import tweepy
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import re 
import pandas as pd
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
import csv
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
n_words= set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud,STOPWORDS
porter = PorterStemmer()

lemmatizer = WordNetLemmatizer() 
nltk.download('punkt')
nltk.download('wordnet'
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import tweepy
api_key = 'ucVkNtONJYEO6dYe8vfYaS9uE'
api_key_secret = '2xakPRdl1Xruv7wQJux2h921KD14iS5679Xv5kRCbwZbh4P3Wv'
access_token = '1408462079598632963-ZF9gAqaqBhLuz7VYWlMC7AiurzqRH0'
access__token_secret = 'YJNu4q3RylxsGVStxcjIGzILxcyWfyrGw5GShV5nx76XT'

In [3]:
auth_handler = tweepy.OAuthHandler(consumer_key=api_key, consumer_secret=api_key_secret)
auth_handler.set_access_token(access_token, access__token_secret)

api = tweepy.API(auth_handler)

In [4]:
search_term = 'bored apes'
tweet_amount = 1000

In [5]:
def clean(text):
  # removing @ tags and links from the text
  text= ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", text).split()) 
  # converting all letters to lower case and relacing '-' with spaces.
  text= text.lower().replace('-', ' ')
  # removing stowards and numbers
  table= str.maketrans('', '', string.punctuation+string.digits)
  text= text.translate(table)
  # tokenizing words
  tokens = word_tokenize(text)
  # stemming the words 
  stemmed = [porter.stem(word) for word in tokens]
  words = [w for w in stemmed if not w in n_words]
  # lemmetizing words
  # lemme= [lemmatizer.lemmatize(word) for word in tokens]
  # words = [w for w in lemme if not w in n_words]
  text = ' '.join(words)
  return text 

In [6]:
def wordcloud_draw(data, color = 'black'):
    words = ' '.join(data)
    cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word  # double check for nay links
                                and not word.startswith('#')  # removing hash tags
                                and word != 'rt'  
                            ])
    wordcloud = WordCloud(stopwords=STOPWORDS, # using stopwords provided by Word cloud its optional since we already removed stopwords :)
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(cleaned_word)
    # using matplotlib to display the images in notebook itself.
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [7]:
def output(search_term): 
  tweet_amount = 1000
  tweets = tweepy.Cursor(api.search, q= search_term, lang = 'en',wait_on_rate_limit=True).items(tweet_amount)
  tweetss=[]
  for tweet in tweets:
      tweet= clean(tweet.text)
      analysis = TextBlob(tweet)
      senti= analysis.sentiment.polarity
      if senti<0 :
        emotion = "NEG"
      elif senti>0:
        emotion= "POS"
      else:
        emotion= "NEU"
      tweetss.append((tweet, senti, emotion))
  df= pd.DataFrame(tweetss, columns= ['tweets', 'senti', 'emotion'])
  # droping retweets
  df= df.drop_duplicates()
  # saving to CSV file
  df.to_csv('data.csv', index= False)
  df_neu= df[ df['emotion'] == 'NEU']
  df_neu= df_neu['tweets']
  df_pos = df[ df['emotion'] == 'POS']
  df_pos = df_pos['tweets']
  df_neg = df[ df['emotion'] == 'NEG']
  df_neg = df_neg['tweets']
  ratio_pos= len(df_pos)/len(df)
  ratio_neg= len(df_neg)/len(df)
  ratio_neu= len(df_neu)/len(df)

  percent_pos = ratio_pos * 100
  percent_neg = ratio_neg * 100 
  percent_neu = ratio_neu * 100 
  return [percent_pos, percent_neg, percent_neu]

In [8]:
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [9]:
import anvil.server
anvil.server.connect('server_57AQBXHXBXQB3SV63LDEGSMA-UP4OJMY33G3H35TG')

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as SERVER


In [10]:
@anvil.server.callable
def pos(search_term): 
  pos_result = output(search_term)
  return pos_result[0]

In [11]:
pos('try')

34.21052631578947

In [14]:
@anvil.server.callable
def neg(search_term): 
  neg_result = output(search_term)
  return neg_result[1]

In [15]:
@anvil.server.callable
def neu(search_term): 
  neu_result = output(search_term)
  return neu_result[2]